# Quadball game-state / logic test

In [1]:
# reload automatically when packages change: jupyter notebook only
%load_ext autoreload
%autoreload 2


In [2]:
# Section 1 — Imports & Test Utilities

import sys
import os
import time
import math
import json
from typing import Dict, Tuple, Optional

import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Small drawing helpers

def clamp(x, a, b):
    return max(a, min(b, x))

def draw_circle(ax, x, y, r=10, color='C0', alpha=0.9, fill=True, **kwargs):
    c = plt.Circle((x, y), r, color=color, alpha=alpha, fill=fill, **kwargs)
    ax.add_patch(c)

def draw_arrow(ax, x, y, dx, dy, color='k'):
    ax.arrow(x, y, dx, dy, head_width=8, head_length=8, fc=color, ec=color)

def pretty(s):
    print(json.dumps(s, indent=2))


In [3]:
# Section 2 — Adjust sys.path & Load Project Modules

# Ensure repo root is on sys.path so `quadball` package imports work when running the notebook
repo_root = os.path.abspath('.')  # notebook placed at repo root
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print('repo_root:', repo_root)

# Try to import project modules
try:
    from core.game_state import GameState
    from core.entities import Player, Ball, Hoop, Vector2, PlayerRole, BallType
    from systems.game_logic_system import GameLogicSystem
    print('Imported quadball modules successfully')
except Exception as e:
    print('Failed to import quadball modules — make sure working directory is repo root or install package editable')
    raise


repo_root: c:\Users\peter\OneDrive\Q4A\quadball
Imported quadball modules successfully


In [106]:
# Section 3 — Create Test GameState & GameLogicSystem

# Create a fresh GameState and GameLogicSystem for testing
state = GameState()
state.add_player(Player(id=1, team=0, position=Vector2(10, 10), velocity=Vector2(0, 0), role=PlayerRole.CHASER))
state.add_player(Player(id=2, team=1, position=Vector2(20, 10), velocity=Vector2(0, 0), role=PlayerRole.CHASER))
game_logic = GameLogicSystem(state)




In [116]:
state.players[1].direction = Vector2(1, 0)
state.players[2].direction = Vector2(-1, 0)
game_logic.update(0.05)
print('player 1 position', state.players[1].position)
print('player 2 position', state.players[2].position)
print('player 1 velocity', state.players[1].velocity)
print('player 2 velocity', state.players[2].velocity)

normal 1.0 0.0
pre-collision player vel 2.0 0.0
pre-collision other player vel -2.0 0.0
dot player 2.0
dot other -2.0
player vel along normal 2.0 0.0
player vel perp 0.0 0.0
other vel along normal -2.0 -0.0
other vel perp 0.0 0.0
combined vel along normal 0.0 0.0
payer new vel 0.0 0.0
other new vel 0.0 0.0
player 1 position Vector2(x=14.807437499999999, y=10.0)
player 2 position Vector2(x=15.1925625, y=10.0)
player 1 velocity Vector2(x=0.0, y=0.0)
player 2 velocity Vector2(x=0.0, y=0.0)


In [89]:
a = Vector2(10, 10)
n = Vector2(0.7, 0.7)
print(a.reflect(n, 0))

Vector2(x=-9.599999999999998, y=-9.599999999999998)


In [6]:
print(a.x * n.x + a.y * n.y)  # Dot product

10
